## Getting the data into a Dataframe

In [35]:
import pandas as pd

canD = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

df = pd.DataFrame(canD[0])

In [36]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [37]:
df.shape

(180, 3)

## Removing Not assigned rows

In [38]:
df=df[df.Borough != 'Not assigned']

In [39]:
df=df.reset_index()
df=df.drop(['index'], axis = 1)

In [40]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Shape of dataframe

In [42]:
df.shape

(103, 3)

## Loading Geolocation

In [46]:
postal = pd.read_csv('http://cocl.us/Geospatial_data')
postal.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the two dataframes

In [47]:
toronto_df = pd.merge(df,postal, on= 'Postal Code')
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [48]:
toronto_df.shape

(103, 5)

## Getting the location of Toronto

In [49]:
from geopy.geocoders import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Importing Folium

In [50]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

## Visualizing the neighborhood

In [51]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [52]:
CLIENT_ID = 'XFKLLHHR0KDWYHQ02QMLOKFVUBF2UDDANUA1JIHDMENMPQLS' # your Foursquare ID
CLIENT_SECRET = 'U4CAR1RBF5MWG3ZAY2KCGWOEOIAVPLSSJ3ERV5F53XGOC1TM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XFKLLHHR0KDWYHQ02QMLOKFVUBF2UDDANUA1JIHDMENMPQLS
CLIENT_SECRET:U4CAR1RBF5MWG3ZAY2KCGWOEOIAVPLSSJ3ERV5F53XGOC1TM


## Exploring the first neighborhood of the dataframe

In [53]:
toronto_df.loc[0, 'Neighborhood']

'Parkwoods'

In [54]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


#### Top 100 venues that are in Marble Parkwoods within a radius of 500 meters

In [67]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XFKLLHHR0KDWYHQ02QMLOKFVUBF2UDDANUA1JIHDMENMPQLS&client_secret=U4CAR1RBF5MWG3ZAY2KCGWOEOIAVPLSSJ3ERV5F53XGOC1TM&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [68]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed6b2ed0de0d9001b755893'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [69]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [71]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## Exploring nearby venues in Toronto

In [72]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [73]:
print(toronto_venues.shape)
toronto_venues.head()

(425, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [74]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North",5,5,5,5,5,5
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",5,5,5,5,5,5
Berczy Park,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",5,5,5,5,5,5
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",5,5,5,5,5,5


## Let's analyze each neighborhood

In [77]:
#Analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Museum,Neighborhood,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [78]:
toronto_onehot.shape

(425, 150)

In [79]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Stadium,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Museum,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.200000,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.2,0.000000,0.2,0.00,0.0,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.20,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.200000,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.200000,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.40,0.000000,0.0,0.0,0.00,0.20,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.400000,0.00,0.0,0.0,0.00,

In [80]:
toronto_grouped.shape

(96, 150)

## Each neighborhood along with the top 5 most common venues

In [81]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2  Latin American Restaurant   0.2
3             Clothing Store   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
           venue  freq
0    Pizza Place   0.4
1            Gym   0.2
2            Pub   0.2
3    Coffee Shop   0.2
4  Movie Theater   0.0


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop   0.4
1  Deli / Bodega   0.2
2     Restaurant   0.2
3    Bridal Shop   0.2
4    Yoga Studio   0.0


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1  Japanese Restaurant  0.25
2                 Café  0.25
3   Chinese Restaurant  0.25
4        Moving Target  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0                Café   0.2
1          Restaurant   0.2
2   Indian Restaurant   0.2
3  Italian Restaurant   0.2
4     Thai Restaurant

In [82]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Creating the new dataframe and displaying the top 10 venues for each neighborhood.

In [85]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Clothing Store,Lounge,Skating Rink,Latin American Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Eastern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pub,Women's Store,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bridal Shop,Deli / Bodega,Restaurant,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
4,"Bedford Park, Lawrence Manor East",Thai Restaurant,Restaurant,Italian Restaurant,Café,Indian Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


# Clustering of Neighborhoods

In [92]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 4, 1, 1, 4, 1, 4, 1], dtype=int32)

## Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [102]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Women's Store,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Portuguese Restaurant,French Restaurant,Hockey Arena,Women's Store,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Breakfast Spot,Coffee Shop,Distribution Center,Spa,Bakery,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Boutique,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store,Clothing Store,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Park,Coffee Shop,Creperie,Italian Restaurant,Sushi Restaurant,Women's Store,Diner,Donut Shop,Dog Run,Distribution Center


In [103]:
toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)

In [104]:
toronto_merged = toronto_merged.astype({"Cluster Labels": int})

# Visualizing the clustered neighborhood

In [105]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining each cluster

### Cluster 1

In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Food & Drink Shop,Women's Store,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
21,York,0,Park,Women's Store,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Department Store
35,East York,0,Park,Pizza Place,Convenience Store,Coffee Shop,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
64,York,0,Convenience Store,Park,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Department Store
66,North York,0,Convenience Store,Park,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Department Store
85,Scarborough,0,Playground,Park,Coffee Shop,Women's Store,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
91,Downtown Toronto,0,Park,Playground,Trail,Women's Store,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


### Cluster 2

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Coffee Shop,Portuguese Restaurant,French Restaurant,Hockey Arena,Women's Store,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store
2,Downtown Toronto,1,Breakfast Spot,Coffee Shop,Distribution Center,Spa,Bakery,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
3,North York,1,Boutique,Vietnamese Restaurant,Coffee Shop,Furniture / Home Store,Clothing Store,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center
6,Scarborough,1,Fast Food Restaurant,Women's Store,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Department Store
7,North York,1,Caribbean Restaurant,Restaurant,Gym,Gym / Fitness Center,Japanese Restaurant,Clothing Store,Italian Restaurant,Discount Store,Café,Donut Shop
9,Downtown Toronto,1,Comic Shop,Clothing Store,Plaza,Theater,Tea Room,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store
11,Etobicoke,1,Filipino Restaurant,Women's Store,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Department Store
13,North York,1,Caribbean Restaurant,Restaurant,Gym,Gym / Fitness Center,Japanese Restaurant,Clothing Store,Italian Restaurant,Discount Store,Café,Donut Shop
15,Downtown Toronto,1,Coffee Shop,Restaurant,Creperie,Japanese Restaurant,Dessert Shop,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
18,Scarborough,1,Electronics Store,Bank,Moving Target,Mexican Restaurant,Rental Car Location,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


### Cluster 3

In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Park,Coffee Shop,Creperie,Italian Restaurant,Sushi Restaurant,Women's Store,Diner,Donut Shop,Dog Run,Distribution Center
10,North York,2,Park,Japanese Restaurant,Pub,Metro Station,Women's Store,Dessert Shop,Dog Run,Distribution Center,Discount Store,Diner
12,Scarborough,2,Bar,Women's Store,Diner,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Dessert Shop
14,East York,2,Diner,Park,Beer Store,Curling Ice,Skating Rink,Women's Store,Donut Shop,Dog Run,Distribution Center,Discount Store
16,York,2,Hockey Arena,Park,Field,Trail,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
36,Downtown Toronto,2,Lake,Plaza,Park,Performing Arts Venue,Fried Chicken Joint,Dance Studio,Distribution Center,Discount Store,Diner,Dessert Shop
40,North York,2,Grocery Store,Business Service,Park,Snack Place,Gym / Fitness Center,Liquor Store,Airport,Athletics & Sports,Food Truck,Bank
46,North York,2,Grocery Store,Business Service,Park,Snack Place,Gym / Fitness Center,Liquor Store,Airport,Athletics & Sports,Food Truck,Bank
49,North York,2,Construction & Landscaping,Park,Basketball Court,Bakery,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store
53,North York,2,Grocery Store,Business Service,Park,Snack Place,Gym / Fitness Center,Liquor Store,Airport,Athletics & Sports,Food Truck,Bank


### Cluster 4

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3,Food Service,Baseball Field,Women's Store,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
101,Etobicoke,3,Baseball Field,Women's Store,Diner,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Dessert Shop


### Cluster 5

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,4,Pharmacy,Gastropub,Pizza Place,Gym / Fitness Center,Bank,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store
17,Etobicoke,4,Pharmacy,Pizza Place,Liquor Store,Coffee Shop,Beer Store,Dessert Shop,Donut Shop,Dog Run,Distribution Center,Discount Store
25,Downtown Toronto,4,Café,Coffee Shop,Italian Restaurant,Grocery Store,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Electronics Store
32,Scarborough,4,Pizza Place,Playground,Deli / Bodega,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
37,West Toronto,4,Korean Restaurant,Asian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Discount Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
39,North York,4,Café,Bank,Japanese Restaurant,Chinese Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
50,North York,4,Pizza Place,Furniture / Home Store,Department Store,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
58,Scarborough,4,Café,General Entertainment,College Stadium,Skating Rink,Department Store,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
59,North York,4,Grocery Store,Steakhouse,Indonesian Restaurant,Ramen Restaurant,Café,Dessert Shop,Donut Shop,Dog Run,Distribution Center,Discount Store
63,York,4,Grocery Store,Bus Line,Convenience Store,Pizza Place,Airport Lounge,Diner,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
